In [1]:
import numpy as np
import pandas as pd
from scipy.interpolate import PchipInterpolator
import xarray as xr
import gsw

with open('../woa18/path-to-woa18-files.txt') as f:
    path_woa = f.readline()

In [2]:
# not using decadal data anymore, just the decav or all
decades = {}
for y in np.arange(1965,1975,1): decades[y] = '6574'
for y in np.arange(1975,1985,1): decades[y] = '7584'
for y in np.arange(1985,1995,1): decades[y] = '8594'
for y in np.arange(1995,2005,1): decades[y] = '95A4'  
for y in np.arange(2005,2019,1): decades[y] = 'A5B7' 

seasons = {}
for m in [1,2,3]: seasons[m] = '13'
for m in [4,5,6]: seasons[m] = '14'
for m in [7,8,9]: seasons[m] = '15'
for m in [10,11,12]: seasons[m] = '16'
    
    
depths = np.array([0,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100,125,150,175,200,225,250,275,300,325,350,
       375,400,425,450,475,500,550,600,650,700,750,800,850,900,950,1000,1050,1100,1150,1200,1250,1300,1350,1400,
       1450,1500,1550,1600,1650,1700,1750,1800,1850,1900,1950,2000,2100,2200,2300,2400,2500,2600,2700,2800,2900,
       3000,3100,3200,3300,3400,3500,3600,3700,3800,3900,4000,4100,4200,4300,4400,4500,4600,4700,4800,4900,
       5000,5100,5200,5300,5400,5500])


In [3]:
#load compiled dissolution data
df = pd.read_csv("data/in_situ_rates_compiled.csv")
#add a new column with pressure to dataset, this is later needed for the CANYON-B calculation
df["Pressure"] = gsw.p_from_z(np.array(df["Depth"]*(-1)), np.array(df["Latitude"]))


### import WOA data
#### Data availability:
- temperature (t): 
    - decadal data or decav
    - 0 - 5500 m for annual and seasonal
- salinity (s): 
    - decadal data or decav
    - 0 - 5500 m for annual and seasonal
- oxygen (o): 
    - all years
    - 0 - 5500 m for annual, 0 - 1500 m for seasonal (57 levels)
- phosphate (p): 
    - all years
    - 0 - 5500 m for annual, 0 - 800 m for seasonal (43 levels)
- silicate (i): 
    - all years
    - 0 - 5500 m for annual, 0 - 800 m for seasonal (43 levels)
    
#### how data is chosen
- in general, the upper 1500 m with seasonal data and below the annual average
- for p and i, the upper 800 m is seasonal data and below the annual average

## define functions to import correct data and interpolate
- for the climatology:
    - upper 1500 m (800 m for nuts) is seasonal (if known) 
    - below annual averages
- for standard deviation:
    - if possible, same as the climatology
    - but often times there are not enough measurements that make up the standard deviation (should be at least 5), then I just take the annual data
- special case doxy sd:
    - there are really not many measurements, often at the site of the dissolution experiment none so sd is not available there or not meaningful
    - I therefore take average SD from grid points in the surrounding area that have enough measurements
        - grid points are considered if they have at least 5 measurements at 1500 m depth (assumption is that above that there are at least that number of measurements for most depths)
        - a considered grid point is only 'participating' in the average, if it also has >= 5 measurements at that certain depth
    - nan values are forward filled
        


In [4]:
def import_var_an(var, lat, lon, seas):
    # import annual (y) and seasonal (s) data
    try:
        if (var == 't') or (var == 's'):
            var_y = xr.open_dataset(path_woa+"woa18_decav_"+var+"00_01.nc", decode_times=False).isel(time=0, drop=True)
            var_s = xr.open_dataset(path_woa+"woa18_decav_"+var+seas+"_01.nc", decode_times=False).isel(time=0, drop=True)
        else:
            var_y = xr.open_dataset(path_woa+"woa18_all_"+var+"00_01.nc", decode_times=False).isel(time=0, drop=True)
            var_s = xr.open_dataset(path_woa+"woa18_all_"+var+seas+"_01.nc", decode_times=False).isel(time=0, drop=True)
    except:
        print(var, 'file is missing')

    # find location of measurement site:
    var_y_loc = var_y.sel(lat=lat, lon=lon, method="nearest", tolerance=0.5)
    var_s_loc = var_s.sel(lat=lat, lon=lon, method="nearest", tolerance=0.5)
    
    # get arrays (annual average (an), standard deviation (sd), number of observations (dd))
    # upper 800 (43 levels) from seasonal (if season not known, then it's just the annual data anyway)
    # everything below is from annual data
    if (var == 'p') or (var == 'i'):
        var_an = np.append(var_s_loc[var+'_an'].values[0:43], var_y_loc[var+'_an'].values[43:])
    else:
        var_an = np.append(var_s_loc[var+'_an'].values[0:57], var_y_loc[var+'_an'].values[57:])
        
    return var_an



def import_var_sd(var, lat, lon, seas):
    if (var == 'p') or (var == 'i'):
        print("DON'T DO PHOSPHATE OR SILICATE, RIGHT NOW THIS DOES NOT WORK AND THE DEPTHS DON'T LINE UP")
    
    # import annual (y) and seasonal (s) data
    try:
        if (var == 't') or (var == 's'):
            var_y = xr.open_dataset(path_woa+"woa18_decav_"+var+"00_01.nc", decode_times=False).isel(time=0, drop=True)
            var_s = xr.open_dataset(path_woa+"woa18_decav_"+var+seas+"_01.nc", decode_times=False).isel(time=0, drop=True)
        else:
            var_y = xr.open_dataset(path_woa+"woa18_all_"+var+"00_01.nc", decode_times=False).isel(time=0, drop=True)
            var_s = xr.open_dataset(path_woa+"woa18_all_"+var+seas+"_01.nc", decode_times=False).isel(time=0, drop=True)
    except:
        print(var, 'file is missing')

    # find location of measurement site:
    var_y_loc = var_y.sel(lat=lat, lon=lon, method="nearest", tolerance=0.5)
    var_s_loc = var_s.sel(lat=lat, lon=lon, method="nearest", tolerance=0.5)


    # if for the seasonal data all dd are above 5, we take the sd from there
    if all(i >= 5 for i in var_s_loc[var+'_dd'].values[0:57]):
        print('sd',var, 'from seasonal + annual data')
        var_sd = np.append(var_s_loc[var+'_sd'].values[0:57], var_y_loc[var+'_sd'].values[57:])
 
    # otherwise we try to use the annual sd for entire array
    # but check whether the annual data has enough observations until depth of 1500 m (level 57)
    elif all(i >= 5 for i in var_y_loc[var+'_dd'].values[0:57]):
        print('sd',var, 'from annual data')
        var_sd = var_y_loc[var+'_sd'].values

    # this will mainly apply to the oxygen data
    else:
        print('sd',var, 'this does not work, use the other approach')
     
    # all unvalid sd values (with dd < 5) are nan
    var_sd = np.where(var_y_loc[var+'_dd'].values < 5, np.nan, var_sd)
   
    return var_sd



def interp_sd_for_doxy(lat, lon):
    # get area
    df_o_sel = select_area(df_o_d_enough,lat,lon)

    # make dataframe 
    df_sel = pd.DataFrame(data={'Depth': depths})
    name_LL = []
    for lats in df_o_sel.lat.unique():
        for lons in df_o_sel.lon.unique():

            o_loc = o.sel(lat=lats, lon=lons, method="nearest")

            name_LL.append(str(lats)+str(lons))
            df_sel[str('dd_'+str(lats)+str(lons))] = o_loc.o_dd.values
            df_sel[str('sd_'+str(lats)+str(lons))] = o_loc.o_sd.values

    # make values nan if dd < 5
    for n in name_LL:    
        df_sel['sd_'+n] = np.where(df_sel['dd_'+n] < 5, np.nan, df_sel['sd_'+n])

    # make new column with average of sd columns
    df_sel['sd_avg'] = df_sel[df_sel.columns[pd.Series(df_sel.columns).str.startswith('sd')]].mean(axis=1)

    # fill all nan values with last valid sd
    df_sel['sd_avg'] = df_sel['sd_avg'].fillna(method='ffill')
    
    # get the interpolator
    interp = PchipInterpolator(depths, df_sel['sd_avg'].values, extrapolate=True)
    print('got oxygen area')
    
    return interp
 
    

def select_area(df,lat,lon):    
    df_loc = df[(df.lon > lon-10) & (df.lon < lon+10) & 
                    (df.lat > lat-10) & (df.lat < lat+10)]    
    return df_loc
    

def forward_fill(arr):
    d = pd.DataFrame(arr)
    d = d.fillna(method='ffill')
    return np.array(d[0].values)


def get_interpolators(var, lat, lon, seas, datatype='an'):        
    if datatype == 'sd':
        var_arr = import_var_sd(var, lat, lon, seas)
    else: 
        var_arr = import_var_an(var, lat, lon, seas)
    
    # get rid of nan values
    var_arr = forward_fill(var_arr)
    
    # make interpolator    
    var_interp = PchipInterpolator(depths, var_arr, extrapolate=True)
    
    return var_interp



## add data to dataset

In [5]:
# prepare oxygen dataset
o = xr.open_dataset(path_woa+"woa18_all_o00_01.nc", decode_times=False).isel(time=0, drop=True)
# make df of data at certain depth
o_d = o.isel(depth = 56)
df_o_d = o_d.to_dataframe().reset_index()

# only keep rows were I have enough measurements
df_o_d_enough = df_o_d[df_o_d.o_dd >= 5]

#ignore performance warnings
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


# Location
for lat, lon in zip(df["Latitude"].unique(), df["Longitude"].unique()):
    locat = (df["Latitude"] == lat) & (df["Longitude"] == lon)
        
    for month in df[locat]['Month'].unique():
        # no month given, annual average instead of a season
        if np.isnan(month):
            seas = '00'
        else:
            seas = seasons[month] 
            
        print(df[(locat)]['Source_abbrev'].unique(), lat, lon, seas)
            
        t_interp = get_interpolators('t', lat, lon, seas) 
        df.loc[(locat),["Temp_woa"]] = t_interp(df[locat]['Depth'])
        t_sd_interp = get_interpolators('t', lat, lon, seas, 'sd') 
        df.loc[(locat),["Temp_SD_woa"]] = t_sd_interp(df[locat]['Depth'])
        
        s_interp = get_interpolators('s', lat, lon, seas)                
        df.loc[(locat),["Sal_woa"]] = s_interp(df[locat]['Depth'])        
        s_sd_interp = get_interpolators('s', lat, lon, seas, 'sd')                
        df.loc[(locat),["Sal_SD_woa"]] = s_sd_interp(df[locat]['Depth'])
        
        o_interp = get_interpolators('o', lat, lon, seas)                
        df.loc[(locat),["Doxy_woa"]] = o_interp(df[locat]['Depth'])
        
        o_interp = interp_sd_for_doxy(lat, lon)                
        df.loc[(locat),["Doxy_SD_woa"]] = o_interp(df[locat]['Depth'])
                
        
        # currently not adding standard deviation
        p_interp = get_interpolators('p', lat, lon, seas)                
        df.loc[(locat),["PO4_woa"]] = p_interp(df[locat]['Depth'])
        
        i_interp = get_interpolators('i', lat, lon, seas)                
        df.loc[(locat),["SiOH4_woa"]] = i_interp(df[locat]['Depth'])
        


['P66'] 18.8 -168.5 00
sd t from seasonal + annual data
sd s from seasonal + annual data
got oxygen area
['B67'] 18.82 -168.51 00
sd t from seasonal + annual data
sd s from seasonal + annual data
got oxygen area
['M77'] 23.3 -70.5 13
sd t from seasonal + annual data
sd s from seasonal + annual data
got oxygen area
['M77'] 23.1 -65.2 15
sd t from seasonal + annual data
sd s from seasonal + annual data
got oxygen area
['M77'] 22.8 -55.1 15
sd t from annual data
sd s from annual data
got oxygen area
['M77'] 25.9 -60.3 15
sd t from annual data
sd s from annual data
got oxygen area
['HE78'] 32.37 -55.0 16
sd t from annual data
sd s from annual data
got oxygen area
['T81'] 4.0 -82.0 15
sd t from annual data
sd s from annual data
got oxygen area
['M82'] 0.0 -152.2 00
sd t from seasonal + annual data
sd s from seasonal + annual data
got oxygen area
['T97'] 22.75 -158.0 13
sd t from seasonal + annual data
sd s from seasonal + annual data
got oxygen area
['T97'] 22.75 -158.0 14
sd t from seasona

## Save data 
- in normal csv
- in csv without headers for the Matlab stuff

In [6]:
df['Doxy_SD_woa'].count()

752

In [7]:
df.columns

Index(['Depth', 'Source', 'Source_abbrev', 'Latitude', 'Longitude', 'Sample',
       'Material', 'Rate_sa', 'Rate_error_sa', 'Organics', 'Device',
       'Deployment_d', 'Biogenic', 'Year', 'Comments', 'Mesh', 'Rate_mass',
       'Month', 'Size', 'Fragmentation_pct', 'Rate_error_mass', 'Temp_CDisk4',
       'pH_CDisk4_T25', 'Salinity_CDisk4', 'TA_CDisk4', 'DIC_CDisk4_calc13',
       'Oca_CDisk4_calc13', 'Oar_CDisk4_calc13', 'Omega_CDisk4_calc13',
       'Pressure', 'Temp_woa', 'Temp_SD_woa', 'Sal_woa', 'Sal_SD_woa',
       'Doxy_woa', 'Doxy_SD_woa', 'PO4_woa', 'SiOH4_woa'],
      dtype='object')

In [8]:
df.to_csv("data/in_situ_rates_compiled_withWOA.csv", index=False)

#save another version just with the stuff I need to do CANYON-B
df_canyonb = df[["Latitude", "Longitude", "Depth", "Pressure", 
                 "Temp_woa", "Sal_woa", "Doxy_woa", "Year", "Month"]].copy()
#if I don't know the month, I put January in
df_canyonb.loc[pd.isna(df_canyonb.Month), "Month"] = 1
df_canyonb.to_csv("data/in_situ_rates_compiled_withWOA_forCANYONB.csv", header=False, index=False)